In [1]:
# Auto-Reload Setup für Entwicklung
# Automatisches Neuladen aller Module bei Änderungen
%load_ext autoreload
%autoreload 2

In [2]:
# Imports - werden automatisch neu geladen bei Änderungen
from datetime import datetime
import pandas as pd
from state_provider.state_provider import state_provider

print("✅ Module erfolgreich importiert mit Auto-Reload aktiviert")

✅ Module erfolgreich importiert mit Auto-Reload aktiviert


In [7]:
# Daten laden und parsen
CSV = "data/full_test_export.csv"
df = pd.read_csv(CSV, sep=";")
state = state_provider.parse_data_to_state(df)

print("✅ Daten erfolgreich geparst")

2025-12-07 14:43:19.544 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:19.545 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:19.551 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:19.552 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:19.545 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:19.551 WARNING streamlit.run

Data time range: 2025-09-10 11:53:00 - 2025-09-30 01:16:00
✅ Daten erfolgreich geparst


In [8]:
# Verify queries after cleanup
print("Testing Vitals Query:")
vitals = state_provider.query_data("vitals")
print(f"Vitals rows: {len(vitals)}")

print("\nTesting Lab Query:")
lab = state_provider.query_data("lab")
print(f"Lab rows: {len(lab)}")

print("\nTesting Device Query:")
ecmo = state_provider.query_data("ecmo")
print(f"ECMO rows: {len(ecmo)}")

2025-12-07 14:43:23.138 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:23.138 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:23.142 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:23.142 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:23.145 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:23.145 WARNING streamlit.run

Testing Vitals Query:
Vitals rows: 3387

Testing Lab Query:
Lab rows: 7490

Testing Device Query:
ECMO rows: 236


In [4]:
# Hilfsfunktion: Alle verfügbaren Methoden anzeigen
def show_available_methods():
    methods = [method for method in dir(state_provider) if not method.startswith('_')]
    print("Verfügbare StateProvider Methoden:")
    for i, method in enumerate(methods, 1):
        print(f"  {i:2}. {method}")
    return methods

# Methoden anzeigen
available_methods = show_available_methods()

Verfügbare StateProvider Methoden:
   1. data_manager
   2. get_device_time_ranges
   3. get_lab_form
   4. get_nearest_ecls_time
   5. get_nearest_impella_time
   6. get_record_id
   7. get_respiration_type
   8. get_respiratory_value
   9. get_selected_time_range
  10. get_selected_view
  11. get_state
  12. get_time_of_mcs
  13. get_time_range
  14. get_value_strategy
  15. get_vasoactive_agents_df
  16. get_vitals_value
  17. has_device_past_24h
  18. has_mcs_records_past_24h
  19. has_parsed_data
  20. parse_data_to_state
  21. query_data
  22. query_manager
  23. reset_state
  24. save_state
  25. set_selected_time_range
  26. update_lab_form_field
  27. update_state


In [5]:
timerange = (df["timestamp"].min(), df["timestamp"].max())
timerange

(Timestamp('2025-09-10 11:53:00'), Timestamp('2025-09-30 01:16:00'))

In [11]:
# Test LabAggregator with new QueryManager
from services.value_aggregation.lab_aggregator import LabAggregator
from datetime import date, time

# Ensure we have data loaded
if not state_provider.has_parsed_data():
    print("Loading data...")
    df = pd.read_csv("data/full_test_export.csv", sep=";", encoding="utf-8")
    state_provider.parse_data_to_state(df)

# Test parameters
test_date = date(2025, 9, 13) # Updated date
nearest_time = time(12, 0)

aggregator = LabAggregator(
    state_provider,
    date=test_date,
    record_id="test_record",
    redcap_event_name="test_event",
    redcap_repeat_instrument="labor",
    redcap_repeat_instance=1,
    value_strategy="nearest",
    nearest_time=nearest_time
)

# Test _get_lab_value directly
val = aggregator._get_lab_value("Blutgase arteriell", "PH")
print(f"PH value (nearest to {nearest_time}): {val}")

# Test create_lab_entry
entry = aggregator.create_lab_entry()
print("Lab Entry created:")
print(entry.model_dump(exclude_none=True))


2025-12-07 14:53:06.400 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:53:06.401 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:53:06.401 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:53:06.401 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
/Users/felixsiegmeier/Documents/Code/python/redcap-mcs/state_provider/query_manager.py:24: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the groupi

PH value (nearest to 12:00:00): 7.358
Lab Entry created:
{'record_id': 'test_record', 'redcap_event_name': 'test_event', 'redcap_repeat_instrument': 'labor', 'redcap_repeat_instance': 1, 'na_post_2': 1, 'ecmella_2': 0, 'assess_time_point_labor': 1, 'assess_date_labor': datetime.date(2025, 9, 13), 'date_assess_labor': datetime.date(2025, 9, 13), 'time_assess_labor': datetime.time(12, 0), 'art_site': 7, 'ph': 7.358, 'inr': 1.6, 'post_act': 0, 'post_crp': 0, 'post_pct': 0, 'hemolysis': 0, 'labor_complete': 0}


In [ ]:
# Check available dates and PH values
lab_df = state_provider.query_data("lab")
if not lab_df.empty:
    print("Date range:", lab_df["timestamp"].min(), lab_df["timestamp"].max())
    ph_data = lab_df[lab_df["parameter"] == "PH"]
    if not ph_data.empty:
        print("PH data samples:")
        print(ph_data[["timestamp", "value"]].head())
        # Pick a date from PH data
        sample_date = ph_data["timestamp"].iloc[0].date()
        print(f"Using sample date: {sample_date}")
    else:
        print("No PH data found.")
else:
    print("No lab data found.")


2025-12-07 14:52:55.160 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:52:55.161 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Date range: 2025-09-10 15:23:00 2025-09-30 00:53:00
PH data samples:
              timestamp  value
98  2025-09-13 10:53:00  7.177
107 2025-09-13 12:23:00  7.358
126 2025-09-13 12:53:00  7.329
216 2025-09-13 16:23:00  7.332
283 2025-09-13 17:23:00  7.288
Using sample date: 2025-09-13


In [24]:
state = state_provider.get_state()
impella = state_provider.query_data("Impella A. axilliaris rechts")
impella

2025-12-10 06:08:57.646 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-10 06:08:57.646 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-10 06:08:57.646 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-10 06:08:57.647 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-10 06:08:57.646 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-10 06:08:57.646 WARNING streamlit.run

,timestamp,source_type,category,parameter,value,source_header,time_range,application,stop,concentration,rate
0,2025-09-15 20:55:00,Impella A. axilliaris rechts,Impella A. axilliaris rechts 2,Zustand,Impella implantiert,Impella A. axilliaris rechts,NaN,NaN,NaN,NaN,NaN
1,2025-09-15 20:55:00,Impella A. axilliaris rechts,Impella A. axilliaris rechts 2,Impella A. axilliaris rechts,Impella A. axilliaris rechts\nVon: 15.09.202...,Impella A. axilliaris rechts,NaN,NaN,NaN,NaN,NaN
2,2025-09-15 20:55:00,Impella A. axilliaris rechts,Impella A. axilliaris rechts 2,HZV (l/min),3.4,Impella A. axilliaris rechts,NaN,NaN,NaN,NaN,NaN
3,2025-09-15 20:55:00,Impella A. axilliaris rechts,Impella A. axilliaris rechts 2,Beobachtung,blutig,Impella A. axilliaris rechts,NaN,NaN,NaN,NaN,NaN
4,2025-09-15 20:55:00,Impella A. axilliaris rechts,Impella A. axilliaris rechts 2,HZV (l/min),ultraschallgestützt (TEE),Impella A. axilliaris rechts,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
87,2025-09-19 14:29:00,Impella A. axilliaris rechts,Impella A. axilliaris rechts 4,Beobachtung,Verband trocken,Impella A. axilliaris rechts,NaN,NaN,NaN,NaN,NaN
88,2025-09-19 14:29:00,Impella A. axilliaris rechts,Impella A. axilliaris rechts 4,HZV (l/min),2.2,Impella A. axilliaris rechts,NaN,NaN,NaN,NaN,NaN
89,2025-09-19 14:29:00,Impella A. axilliaris rechts,Impella A. axilliaris rechts 4,Purgedruck in mmHg,513.0,Impella A. axilliaris rechts,NaN,NaN,NaN,NaN,NaN
90,2025-09-19 14:29:00,Impella A. axilliaris rechts,Impella A. axilliaris rechts 4,Purgefluß in ml/h,6.6,Impella A. axilliaris rechts,NaN,NaN,NaN,NaN,NaN


In [23]:
df = pd.read_csv("data/full_test_export.csv", sep=";", encoding="utf-8")
df["source_type"].unique()

array(['PatientInfo', 'Vitals', 'Vitalparameter (manuell)', 'Ereignis',
       'Versorgungsstatus/Pflegebedarf', 'Zugänge', 'Allergien',
       'regelmäßige Besuche', 'Kostform', 'Körperlicher Zustand',
       'Ernährung', 'Sturzrisikoskala nach Huhn', 'Körperpflege',
       'Patientenverfügung', 'Kontaktperson',
       'Ernährungsstatus - Vorscreening', 'Patientenhilfsmittel',
       'Bewußtsein', 'Chronischer Schmerzpatient', 'Grösse/Gewicht',
       'bekannte Risiken', 'Sprache', 'Besonderheiten', 'Mobilität',
       'Diabetes', 'Covid-Impfung Status', 'Ausscheiden', 'Pflegenotizen',
       'Hautinspektion', 'Körper', 'Mobilität und Bewegung',
       'Soziales Umfeld', 'Berufsgruppe', 'Dekubitusrisiko', 'Lab',
       'Arzt Verlauf', 'Medication',
       'peripherer Venenzugang Ellenbeuge links', 'PPR Zusatzitems E',
       'Richmond-Agitation-Sedation', 'Anordnung nach ärztl. Rücksprache',
       'Maßnahmen Mobilisation', 'Prophylaxen',
       'Maßnahmen Bewegungstherapie', 'Einschä